In [1]:
import snscrape.modules.twitter as tweet
import pandas as pd
import datetime

# Get Tweets

query1 = "Northeastern events"
query2 = "Northeastern oncampusemployment"
query3 = "Northeastern Co-op Opportunities"
query4 = "Northeastern University"

querylist = [query1, query2, query3, query4]
li = []

for i in querylist:
    count = 0
    for t in tweet.TwitterHashtagScraper(i).get_items():
        tweet_map={}
        a = vars(t)
        #print (a.keys())
        #print (a['content']+ " ====>"+ a['url'])
        tweet_map['TwitterID']=str(a['id'])
        tweet_map['TwitterHandle']=a['user']
        tweet_map['TwitterText']=a['content']
        tweet_map['HashTags']=a['hashtags']
        tweet_map['CreatedAt']=a['date']
        tweet_map['likes'] = a['likeCount']
        tweet_map["replies"] = a['replyCount']
        tweet_map['retweet'] = a["retweetCount"]
        tweet_map['Query']= i
        li.append(tweet_map)

        if count==50:
            break
        count+=1
    
df = pd.DataFrame(li)
df.to_csv("Tweets.csv",index=False)


import mysql.connector
conn = mysql.connector.connect(host='localhost',
                        database='mastersstudents',
                        user='root',
                        password='root')

cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("SELECT DATABASE()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)
cursor = conn.cursor()


df = pd.read_excel("D:\Darshana\\NEU\Study\DMDD\Project\Mstudents.xlsx", sheet_name='Tweets')

def get_user_details(username):
    user_detail = {}
    user = tweet.TwitterUserScraper(username)
    user = user.entity
    user_detail["full_name"] = user.displayname
    user_detail["username"] = user.username
    user_detail["profile_image"] = user.profileImageUrl
    user_detail["verified"] = user.verified
    user_detail["id"] = user.id
    user_detail["description"] = user.description
    user_detail["createdAt"] = user.created
    user_detail["followers"] = user.followersCount
    user_detail["following"] = user.friendsCount
    return user_detail

c = 0
for i, row in df.iterrows():
    try:
        user = row['TwitterHandle'].replace("https://twitter.com/", "")
        user_detail = get_user_details(user)

        query = """insert into twitter(TwitterID, TwitterHandle, UserId, TwitterText,HashTags,CreatedAt,Likes, retweetCount,replyCount,Query) values ({0}, "{1}", {2}, "{3}", "{4}", "{5}" ,{6}, {7}, {8}, "{9}")""".format(int(row['TwitterID']), row['TwitterHandle'], user_detail['id'],row['TwitterText'].replace('\t', '.'), row['HashTags'], row['CreatedAt'], row["likes"], row["retweet"], row["replies"],row['Query'])
        cursor.execute(query)
        
        hashtags = eval(row['HashTags'])
        
        for h in hashtags:
            cursor.execute("""insert into tweet_tags(TwitterID, HashTags) values({0}, "{1}")""".format(row['TwitterID'], h))
        
        query = """insert into TwitterUser(userid, UserName, handle, Name,CreatedAt, Followers_Count,FRIENDS_Count, Account_Verified, Description, Profile_image_url) values ({0}, "{1}", "{2}", "{3}", "{4}", {5}, {6}, "{7}", "{8}", "{9}")""".format(int(user_detail['id']), user_detail['username'], row["TwitterHandle"],user_detail['full_name'], user_detail['createdAt'], int(user_detail['followers']), int(user_detail['following']), user_detail["verified"], user_detail["description"], user_detail["profile_image"])
        cursor.execute(query)

    except Exception as e:
        print (e)
        c+=1

print (c)
conn.commit()


In [ ]:
df = pd.read_excel("D:\Darshana\\NEU\Study\DMDD\Project\Mstudents.xlsx", sheet_name='StudentClub')

c = 0
for i, row in df.iterrows():
    try:
    #     print (int(row['TwitterID']))

        #query = """insert into twitter(TwitterID, TwitterHandle, TwitterText,HashTags,CreatedAt,Query) values ({0}, "{1}", "{2}", "{3}", "{4}", "{5}")""".format(int(row['TwitterID']), row['TwitterHandle'], row['TwitterText'].replace('\t', '.'), row['HashTags'], row['CreatedAt'], row['CreatedAt'])

#         query = """INSERT INTO STUDENT (StudentID, StudentName, Gender, Course, College, FeePayment, CreditsEarned, IntakeYear) VALUES({0}, "{1}", "{2}", "{3}", "{4}", "{5}", {6},{7});""".format(int(row['StudentID']), row['StudentsName'], row['Gender'],row['Course'] ,row['College'], row['FeePayment'],row['CreditsEarned'], row['IntakeYear']    
        
        
#         query = """INSERT INTO College (CollegeName, DeptName, DeptID, DeanName) VALUES("{0}", "{1}", {2}, "{3}")""".format(row['CollegeName'], row['DeptName'], row['DeptID'],row['DeanName'])
#         print (int(row['JobId']), row['Employer'], row['Location'], row['SkillsRequired'],int(row['Salary']),int(row['Duration'].replace(" Months", "")), row['PostingDate'], row['Deadline'], int(row['Vacancy']))
        
#         query = """INSERT INTO OnCampusEmployment(JobID, Employer, Location, SkillsRequired, SalaryInUSD, DurationInMonths, PostingDate, Deadline, Vacancy) VALUES({0}, "{1}","{2}","{3}",{4},{5},"{6}","{7}",{8})""".format(int(row['JobId']), row['Employer'], row['Location'], row['SkillsRequired'],int(row['Salary']),int(row['Duration'].replace(" Months", "")), datetime.datetime.strptime(row['PostingDate'].replace(" EST", ""), "%a, %b %d, %Y %I:%M %p"), datetime.datetime.strptime(row['Deadline'].replace(" EST", ""), "%a, %b %d, %Y %I:%M %p"), int(row['Vacancy'])) 
        
    
#         query = """INSERT INTO InternshipsAndCoop(JobID, Company, Location, SkillsRequired, Salary, Duration, PostingDate, Deadline, Position) VALUES({0},"{1}","{2}","{3}",{4},{5},"{6}","{7}","{8}")""".format(int(row['JobId']), row['Company'], row['Location'],row['SkillsRequired'], int(row['Salary']),int(row['Duration']), datetime.datetime.strptime(row['PostingDate'].replace(" EST", ""), "%a, %b %d, %Y %I:%M %p"), datetime.datetime.strptime(row['Deadline'].replace(" EST", ""), "%a, %b %d, %Y %I:%M %p"),row['Position'])
        
#         query = """INSERT INTO EventDetails(EID, Ename, Location, Purpose, Dateandtime, FareinUSD) VALUES({0},"{1}","{2}","{3}","{4}","{5}")""".format(int(row['EID']), row['Ename'],row['Location'],row['Purpose'],datetime.datetime.strptime(row['Dateandtime'].replace(" EST", ""), "%a, %b %d, %Y %I:%M %p"), row['FareinUSD'])
#         print (query)
    
        query = """INSERT INTO StudentClub(ID, Name, Head, Contacts, Motive) VALUES({0},"{1}","{2}","{3}","{4}")""".format(int(row['ID']), row['Name'], row['Head'], row['Contacts'], row['Motive'])
        cursor.execute(query)

    except Exception as e:
        print (e)
        c+=1
print (c)
conn.commit()